# Load Spectorgram Images into DataLoader for Training

In [2]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import pandas as pd

C:\Users\cassieb\Anaconda3\envs\pytorchaudio\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [5]:
data_path = './data/spectrograms' #looking in subfolder train

yes_no_dataset = datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()
                                  ])
)
print(yes_no_dataset)
print(yes_no_dataset[5][0].size())

Dataset ImageFolder
    Number of datapoints: 7985
    Root location: ./data/spectrograms
    StandardTransform
Transform: Compose(
               Resize(size=(201, 81), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )
torch.Size([3, 201, 81])


In [6]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(yes_no_dataset))
test_size = len(yes_no_dataset) - train_size
yes_no_train_dataset, yes_no_test_dataset = torch.utils.data.random_split(yes_no_dataset, [train_size, test_size])

print(len(yes_no_train_dataset))
print(len(yes_no_test_dataset))

6388
1597


In [7]:
train_dataloader = torch.utils.data.DataLoader(
    yes_no_train_dataset,
    batch_size=5,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    yes_no_test_dataset,
    batch_size=5,
    num_workers=2,
    shuffle=True
)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))


Using cuda device


In [10]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)
    
model = CNNet().to(device)

print(model)

CNNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (flatten): Flatten()
  (fc1): Linear(in_features=51136, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)


In [11]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [12]:
epochs = 5

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------
loss: 0.658943  [    0/ 6388]
loss: 0.666945  [  500/ 6388]
loss: 0.721330  [ 1000/ 6388]
loss: 0.686157  [ 1500/ 6388]
loss: 0.723326  [ 2000/ 6388]
loss: 0.696066  [ 2500/ 6388]
loss: 0.668008  [ 3000/ 6388]
loss: 0.678210  [ 3500/ 6388]
loss: 0.692726  [ 4000/ 6388]
loss: 0.679610  [ 4500/ 6388]
loss: 0.406854  [ 5000/ 6388]
loss: 0.591798  [ 5500/ 6388]
loss: 0.548504  [ 6000/ 6388]

Test Error:
acc: 79.8%, avg loss: 0.089978

Epoch 2
-------------------------------
loss: 0.507070  [    0/ 6388]
loss: 0.435476  [  500/ 6388]
loss: 0.318111  [ 1000/ 6388]
loss: 0.168022  [ 1500/ 6388]
loss: 0.316531  [ 2000/ 6388]
loss: 0.330804  [ 2500/ 6388]
loss: 0.420686  [ 3000/ 6388]
loss: 0.127864  [ 3500/ 6388]
loss: 0.387181  [ 4000/ 6388]
loss: 0.351763  [ 4500/ 6388]
loss: 0.845003  [ 5000/ 6388]
loss: 0.304001  [ 5500/ 6388]
loss: 0.421326  [ 6000/ 6388]

Test Error:
acc: 90.2%, avg loss: 0.049939

Epoch 3
-------------------------------
loss: 0.11

In [13]:
size = len(test_dataloader.dataset)
model.eval()
test_loss, correct = 0, 0

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        print(f"predicted: {pred.argmax(1)}")
        print(f"actual: {Y}")
        test_loss += cost(pred, Y).item()
        #print(pred.argmax(axis=1))
        #print(Y.argmax(axis=1))
        #correct += (pred.argmax(1) == Y.argmax(1)).type(torch.float).sum().item()
        correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

predicted: tensor([0, 0, 0, 1, 1], device='cuda:0')
actual: tensor([0, 0, 0, 1, 1], device='cuda:0')
predicted: tensor([0, 1, 1, 1, 1], device='cuda:0')
actual: tensor([0, 1, 1, 1, 0], device='cuda:0')
predicted: tensor([0, 1, 0, 1, 1], device='cuda:0')
actual: tensor([0, 1, 0, 1, 1], device='cuda:0')
predicted: tensor([1, 0, 1, 0, 0], device='cuda:0')
actual: tensor([1, 0, 1, 0, 0], device='cuda:0')
predicted: tensor([0, 1, 1, 0, 1], device='cuda:0')
actual: tensor([0, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 0], device='cuda:0')
actual: tensor([0, 1, 0, 0, 0], device='cuda:0')
predicted: tensor([0, 0, 1, 0, 1], device='cuda:0')
actual: tensor([0, 0, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 0, 0, 0, 1], device='cuda:0')
actual: tensor([0, 0, 0, 0, 1], device='cuda:0')
predicted: tensor([1, 1, 0, 0, 0], device='cuda:0')
actual: tensor([1, 1, 0, 0, 0], device='cuda:0')
predicted: tensor([0, 0, 0, 0, 1], device='cuda:0')
actual: tensor([0, 0, 0, 0, 1], device=

predicted: tensor([1, 1, 1, 0, 1], device='cuda:0')
actual: tensor([1, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 1, 0, 1], device='cuda:0')
actual: tensor([0, 1, 1, 1, 1], device='cuda:0')
predicted: tensor([1, 0, 1, 1, 1], device='cuda:0')
actual: tensor([1, 0, 1, 1, 1], device='cuda:0')
predicted: tensor([0, 0, 1, 0, 1], device='cuda:0')
actual: tensor([0, 0, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 1], device='cuda:0')
actual: tensor([0, 1, 0, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 1, 0, 1], device='cuda:0')
actual: tensor([0, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([1, 0, 0, 0, 0], device='cuda:0')
actual: tensor([1, 0, 0, 0, 1], device='cuda:0')
predicted: tensor([0, 0, 0, 1, 0], device='cuda:0')
actual: tensor([0, 0, 0, 1, 0], device='cuda:0')
predicted: tensor([1, 1, 0, 1, 0], device='cuda:0')
actual: tensor([1, 1, 0, 1, 0], device='cuda:0')
predicted: tensor([0, 0, 1, 0, 0], device='cuda:0')
actual: tensor([0, 0, 1, 0, 0], device=

actual: tensor([0, 1, 0, 0, 0], device='cuda:0')
predicted: tensor([0, 0, 1, 0, 0], device='cuda:0')
actual: tensor([0, 0, 1, 0, 0], device='cuda:0')
predicted: tensor([0, 1, 1, 1, 0], device='cuda:0')
actual: tensor([0, 1, 1, 1, 0], device='cuda:0')
predicted: tensor([1, 1, 0, 1, 0], device='cuda:0')
actual: tensor([1, 1, 0, 1, 0], device='cuda:0')
predicted: tensor([1, 1, 0, 1, 1], device='cuda:0')
actual: tensor([1, 1, 0, 1, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 1], device='cuda:0')
actual: tensor([0, 1, 0, 0, 1], device='cuda:0')
predicted: tensor([1, 1, 1, 0, 1], device='cuda:0')
actual: tensor([1, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([1, 0, 0, 1, 0], device='cuda:0')
actual: tensor([1, 0, 0, 1, 0], device='cuda:0')
predicted: tensor([0, 0, 0, 0, 0], device='cuda:0')
actual: tensor([0, 0, 0, 1, 0], device='cuda:0')
predicted: tensor([1, 1, 1, 0, 0], device='cuda:0')
actual: tensor([1, 1, 1, 0, 0], device='cuda:0')
predicted: tensor([0, 0, 0, 0, 1], device=

predicted: tensor([1, 1, 0, 1, 0], device='cuda:0')
actual: tensor([1, 1, 0, 1, 0], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 0], device='cuda:0')
actual: tensor([0, 1, 0, 1, 0], device='cuda:0')
predicted: tensor([1, 1, 1, 0, 1], device='cuda:0')
actual: tensor([1, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 1, 0, 1], device='cuda:0')
actual: tensor([0, 0, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 1], device='cuda:0')
actual: tensor([0, 1, 0, 0, 1], device='cuda:0')
predicted: tensor([1, 1, 1, 1, 1], device='cuda:0')
actual: tensor([1, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([1, 0, 1, 1, 1], device='cuda:0')
actual: tensor([1, 0, 1, 1, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 1], device='cuda:0')
actual: tensor([0, 1, 0, 0, 0], device='cuda:0')
predicted: tensor([0, 1, 1, 0, 1], device='cuda:0')
actual: tensor([1, 1, 1, 0, 1], device='cuda:0')
predicted: tensor([0, 1, 0, 0, 1], device='cuda:0')
actual: tensor([0, 1, 0, 0, 1], device=